In [1]:
import os
import re
import json
import psycopg2
from tqdm import tqdm

# Manage API Keys
from dotenv import load_dotenv
load_dotenv()
KEY_POSTGRES = os.getenv("KEY_POSTGRES")

In [2]:
JSON_input_ROR = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\v2.0-2025-12-16-ror-data.json


In [3]:
conn = psycopg2.connect(
    "dbname=ror_institutions user=postgres password=" + KEY_POSTGRES
)
cur = conn.cursor()

with open(JSON_input_ROR) as f:
    data = json.load(f)

# -----------------------------
# 0. Clear all existing data
# -----------------------------
cur.execute("""
    TRUNCATE TABLE institution_relationships CASCADE;
    TRUNCATE TABLE institution_locations CASCADE;
    TRUNCATE TABLE institution_external_ids CASCADE;
    TRUNCATE TABLE institution_domains CASCADE;
    TRUNCATE TABLE institution_names CASCADE;
    TRUNCATE TABLE institutions CASCADE;
""")
conn.commit()


# -----------------------------
# 1. First pass: insert all institution data
# -----------------------------
for org in tqdm(data):

    institution_id = re.sub(r'https://ror.org/', '', org['id'])
    types_array = org.get('types', [])

    # Insert into institutions (now including types array)
    cur.execute("""
        INSERT INTO institutions (
            institution_id,
            status,
            established_year,
            types,
            created_at,
            last_modified,
            created_schema_version,
            modified_schema_version
        )
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (institution_id) DO UPDATE SET
            status = EXCLUDED.status,
            types = EXCLUDED.types
    """, (
        institution_id,
        org['status'],
        org.get('established'),
        types_array,
        org['admin']['created']['date'],
        org['admin']['last_modified']['date'],
        org['admin']['created']['schema_version'],
        org['admin']['last_modified']['schema_version']
    ))

    # -----------------------------
    # Insert names / aliases
    # -----------------------------
    for name in org['names']:
        for ntype in name['types']:
            cur.execute("""
                INSERT INTO institution_names (
                    institution_id,
                    name,
                    normalized_name,
                    lang,
                    type
                )
                VALUES (%s,%s,%s,%s,%s)
                ON CONFLICT DO NOTHING
            """, (
                institution_id,
                name['value'],
                name['value'].lower(),
                name.get('lang'),
                ntype
            ))

    # -----------------------------
    # Insert domains
    # -----------------------------
    for domain in org.get('domains', []):
        cur.execute("""
            INSERT INTO institution_domains (institution_id, domain)
            VALUES (%s,%s)
            ON CONFLICT DO NOTHING
        """, (institution_id, domain))

    # -----------------------------
    # Insert external IDs
    # -----------------------------
    for eid in org.get('external_ids', []):
        cur.execute("""
            INSERT INTO institution_external_ids (
                institution_id,
                external_id_type,
                preferred_value,
                all_values
            )
            VALUES (%s,%s,%s,%s)
            ON CONFLICT DO NOTHING
        """, (
            institution_id,
            eid['type'],
            eid.get('preferred'),
            eid['all']
        ))

    # -----------------------------
    # Insert locations
    # -----------------------------
    for loc in org['locations']:
        g = loc['geonames_details']
        cur.execute("""
            INSERT INTO institution_locations (
                institution_id,
                geonames_id,
                city_name,
                country_code,
                country_name,
                subdivision_code,
                subdivision_name,
                lat,
                lng,
                continent_code,
                continent_name
            )
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            ON CONFLICT DO NOTHING
        """, (
            institution_id,
            loc['geonames_id'],
            g['name'],
            g.get('country_code'),
            g.get('country_name'),
            g.get('country_subdivision_code'),
            g.get('country_subdivision_name'),
            g.get('lat'),
            g.get('lng'),
            g.get('continent_code'),
            g.get('continent_name')
        ))

# Commit after first pass
conn.commit()


# -----------------------------
# 2. Second pass: insert relationships
# -----------------------------
for org in tqdm(data):

    institution_id = re.sub(r'https://ror.org/', '', org['id'])

    for rel in org.get('relationships', []):

        rel_id = re.sub(r'https://ror.org/', '', rel['id'])

        if rel['type'] in ('parent', 'successor'):
            child_id = institution_id
            parent_id = rel_id

        elif rel['type'] in ('child', 'predecessor'):
            child_id = rel_id
            parent_id = institution_id

        else:
            continue

        if child_id and parent_id:
            cur.execute("""
                INSERT INTO institution_relationships (
                    child_institution_id,
                    parent_institution_id,
                    relationship_type,
                    label
                )
                VALUES (%s,%s,%s,%s)
                ON CONFLICT DO NOTHING
            """, (
                child_id,
                parent_id,
                rel['type'],
                rel['label']
            ))

# Commit relationships
conn.commit()

cur.close()
conn.close()

100%|███████████████████████████████████████████████████████████████████████| 120445/120445 [00:08<00:00, 13513.59it/s]
